<a href="https://colab.research.google.com/github/sleepbook53/Final-project/blob/main/03_03%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C(%EC%BD%94%EB%9E%A9ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [4]:
ytb1 = pd.read_csv('/content/drive/MyDrive/K-디지털 품질재단 - 빅데이터 분석/Final project/ytb_input2.csv')

# 자막에서 키워드 추출

In [5]:
okt = Okt()

# 다국어 씨버트 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [6]:
subkeyword = []

In [7]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return subkeyword.append([words[idx] for idx in keywords_idx]), print('o')
  

In [ ]:
for doc in ytb1['subtitle'] : 
  try :
    # 형태소 분석
    tokenized_doc = okt.pos(doc)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

    # 타이그램으로 묶기
    n_gram_range = (2, 3)

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()
    
    # 모델가져오기
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    # 상위 5개 추출
    mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.5)
    print('')
  except :
    subkeyword.append('x')
    print('x')
    


# 파일 저장하기

In [19]:
import csv

with open("subkeyword", 'w') as file:
  writer = csv.writer(file)
  writer.writerow(subkeyword)

In [44]:
ytb1['subkeyword']

0      [수익 부자 재테크, 성공 어머니 실패, 첫걸음 적금 주식, 사업 자체 스토리텔링,...
1       [굳이 종목 사지, 투자 두려움, 시간 월급 투자, 문제 단타 세금, 사람 구매 조사]
2      [수익 월급 저축, 월세 가지 쥐약, 그냥 스포츠 웹툰, 금값 주식, 매월 마다 꼬...
3      [거지 운영 티스토리, 학교 과제, 온라인 쇼핑 사람, 수익 적기 때문, 연금 고향...
4      [다른 재테크 영상, 진짜꿀 혜택, 댓글 반응, 성형수술 가격 발품, 사업자 영리 목적]
                             ...                        
452    [싱크대 청소 양파망, 바느질 보관 마스크, 재활용 꿀팁, 활용 매일 사용, 싱크대...
453    [인플레이션 통제 우려, 긴축 주식 공격, 다른 자산 금융위기, 우려 대두 때문, ...
454    [손실 요즘 담배값, 수익 더욱 긴장, 안녕하십니까 님들, 메일 한마디 시장, 세상...
455    [시장 위축 서울, 연속 하락 세입, 부동산 이조사 둘째, 지난달 첫째 기록, 한국...
456    [반도체 우크라이나 전쟁, 투자자 생각 원금, 오늘 영상, 글로벌 경기 둔화, 삼성...
Name: subkeyword, Length: 457, dtype: object

In [45]:
# 인덱스, 결측치 제거
ytb1= ytb1.drop(ytb1[ytb1['subkeyword'] == 'x'].index)
ytb1 = ytb1.reset_index(drop = True)

In [49]:
# 저장하기
ytb1.to_csv('./ytb_keyword1.csv',index = False)